[`pidgin.weave`](weave.md.ipynb) allows code to be execute with the display function.

In [1]:
    if __name__ == '__main__': 
        %reload_ext pidgin

In [2]:
    import ast, IPython, jsonschema, pyld, traitlets, operator

In [3]:
    def load_ipython_extension(shell: IPython.core.interactiveshell.InteractiveShell):
        shell.ast_transformers.append(Expand())
        shell.events.register('post_run_cell', validate_schema)

In [4]:
    def unload_ipython_extension(shell: IPython.core.interactiveshell.InteractiveShell):
        for i, event in enumerate(shell.events.callbacks['post_run_cell']):
            if event is validate_schema: shell.events.callbacks['post_run_cell'].pop(i); break
        for i in reversed([i for i, object in enumerate(shell.ast_transformers) if isinstance(object, (Expand,))]): shell.ast_transformers.pop(i)
        pops = []
        for i, post in enumerate(shell.events.callbacks['post_run_cell']):
            if post == validate_schema: pops.append(i)
        for i in reversed(pops): shell.events.callbacks['post_run_cell'].pop(i)

In [5]:
    class Expand(ast.NodeTransformer):      
> __annotations__ = {"@context": {'w': "http://en.wikipedia.org/wiki/"}}
> str("w:Thermodynamics")
'http://en.wikipedia.org/wiki/Thermodynamics'
        
        def visit_FunctionDef(Expand, node):
            """explicitly attach docstring"""
            if isinstance(node.body[0], ast.Expr) and isinstance(node.body[0].value, ast.Str):
                assign = ast.increment_lineno(ast.parse("""function._"""+"""_doc_"""+"""_ = expr""").body[0], node.lineno)
                assign.targets[0].value = ast.copy_location(ast.parse(F"{node.name}").body[0].value, node)
                assign.value = ast.copy_location(node.body[0].value, node)
                return [node, assign]
            return node
        visit_ClassDef = visit_FunctionDef
        
        def visit_Str(Expand, node):
            expand = ast.parse("""str(__import__('jason').S(
                '*')@globals().get('__annotations__', {}).get('@context', {}))""").body[0].value
            expand.args[0].left.args[0] = node
            return ast.copy_location(expand, node)

        def visit_AnnAssign(Expand, node): return node
        
        def visit_Assign(Expand, node):
            return all(getattr(object, 'id', '') != '__annotations__'  for object in node.targets) and super().generic_visit(node) or node
        
        def visit_Call(Expand, node):
            try: 
                if operator.attrgetter('value.id')(node.func) == '__annotations__': return node
            finally: return super().generic_visit(node)
            
        def visit_Subscript(Expand, node):
            if isinstance(node.value, ast.Name) and node.value.id == "__annotations__":
                return node
            return super().generic_visit(node)

In [6]:
    def expand(str, context=None):
        if len(str.splitlines()) == 1 and not str.startswith('@'):
            object = pyld.jsonld.expand({'@context': dict(context or {}), str: ''})
            if object: return next(iter(object[0].keys()))
        return str

In [7]:
    def validate_schema(raw_cell, store_history=False, silent=False, shell_futures=True):
        shell = IPython.get_ipython()
        schema = shell.user_ns.get('__schema__', {})
        jsonschema.validate({k:v for k,v in shell.user_ns.items() if k in schema.get('properties', {})}, schema)